In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import keras.metrics
import numpy as np
import joblib
from scipy.stats import reciprocal


In [3]:
def return_model(n_hidden = 1 , n_neurons=30, learning_rate = 0.01, input_shape = X_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu",  activity_regularizer=l1(0.01)))
        model.add(Dropout(0.3))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss= 'mse')
    return model 

In [12]:
sheet = pd.read_excel(r"test2.xlsx")
print(sheet.info())

sheet = sheet.dropna()
features = sheet.iloc[:,1:6]
result = sheet.iloc[:,6:]
print(features)
X_train,X_test, y_train, y_test = train_test_split(features,result.iloc[:,3], test_size = 0.05)
X_train,X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OndernemingID     2100 non-null   int64  
 1   Postcode          2100 non-null   int64  
 2   Beursnotatie      2100 non-null   int64  
 3   AantalWerknemers  1892 non-null   float64
 4   Omzet             1725 non-null   float64
 5   TotaalActiva      2053 non-null   float64
 6   duurzaamheidrank  2100 non-null   float64
 7   governancerank    2100 non-null   float64
 8   environmentrank   2100 non-null   float64
 9   TotalRank         2100 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 164.2 KB
None
      Postcode  Beursnotatie  AantalWerknemers      Omzet  TotaalActiva
0         8520             1              22.0   458778.0    41156846.0
1         3583             1              65.0   123111.0    24339573.0
2         3500             1             133.0   113602.

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_valid = scaler.fit_transform(X_val)
print(X_train.shape[1:])
batch_size = [10, 20, 40, 60, 80, 100, 1000]
epochs = [10, 50, 100, 200, 500]
param_grid = dict(batch_size=batch_size, epochs=epochs)

(5,)


In [25]:
model = KerasRegressor(return_model)
param_dist = {
    "n_hidden":[0,1,2,3,4,5,6],
    "n_neurons":np.arange(1,250),
    "learning_rate":reciprocal(3e-5, 3e-2)
}
#print(y_test)
#print(X_test)
#print(y_train)
rnd_mod = RandomizedSearchCV(model, param_distributions=param_dist, n_iter= 50, cv=5)
rnd_mod.fit(X_train, y_train, epochs= 20, validation_data=(X_test, y_test))

joblib.dump(rnd_mod, 'model.pkl')

[[ 1.40314636e+00 -1.09108945e-01  3.33676171e+00 -6.35209164e-01
   3.32813662e-01]
 [-8.66861667e-01 -1.09108945e-01  1.34224007e-01  3.55915153e-01
  -6.01383615e-01]
 [ 1.18985701e+00 -1.09108945e-01  1.05629920e-01 -6.34902048e-01
   1.75385654e+00]
 [-9.33895461e-01 -1.09108945e-01 -4.37657726e-01  1.96705032e-02
  -4.06286384e-01]
 [-6.74901257e-01 -1.09108945e-01  9.92046607e-01 -6.33197392e-01
  -3.70334231e-01]
 [ 1.20265437e+00 -1.09108945e-01  3.34382613e-01  4.81052284e-01
   3.64553989e-01]
 [-9.88741293e-01 -1.09108945e-01 -3.80469553e-01 -6.36480638e-01
  -5.95241312e-01]
 [-8.76002639e-01 -1.09108945e-01 -6.09222246e-01 -6.36147716e-01
  -3.77205515e-01]
 [-9.43036433e-01 -1.09108945e-01  1.27798747e+00 -6.37212082e-01
  -7.56918358e-01]
 [-4.67705894e-01 -1.09108945e-01 -7.52192680e-01  8.94199945e-02
  -7.91794815e-01]
 [ 1.25079683e+00 -1.09108945e-01  2.50753320e+00  3.14664195e-01
  -2.86091253e-02]
 [-8.51626714e-01 -1.09108945e-01 -8.66569026e-01 -6.88702485e-02

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_6316\2414782382.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(return_model)


37/37 [==============================] - 0s 2ms/step - loss: 2.3567 - val_loss: 2.1708
Epoch 2/20
37/37 [==============================] - 0s 694us/step - loss: 2.3002 - val_loss: 2.1204
Epoch 3/20
37/37 [==============================] - 0s 834us/step - loss: 2.2446 - val_loss: 2.0713
Epoch 4/20
37/37 [==============================] - 0s 722us/step - loss: 2.1910 - val_loss: 2.0235
Epoch 5/20
37/37 [==============================] - 0s 722us/step - loss: 2.1381 - val_loss: 1.9769
Epoch 6/20
37/37 [==============================] - 0s 694us/step - loss: 2.0872 - val_loss: 1.9310
Epoch 7/20
37/37 [==============================] - 0s 694us/step - loss: 2.0369 - val_loss: 1.8867
Epoch 8/20
37/37 [==============================] - 0s 667us/step - loss: 1.9883 - val_loss: 1.8433
Epoch 9/20
37/37 [==============================] - 0s 722us/step - loss: 1.9411 - val_loss: 1.8002
Epoch 10/20
37/37 [==============================] - 0s 722us/step - loss: 1.8942 - val_loss: 1.7586
Epoch 11/20


Epoch 19/20
37/37 [==============================] - 0s 694us/step - loss: 1.0418 - val_loss: 0.8748
Epoch 20/20
10/10 [==============================] - 0s 444us/step - loss: 0.9144
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 2.4843 - val_loss: 3.0710
Epoch 2/20
37/37 [==============================] - 0s 694us/step - loss: 2.4461 - val_loss: 3.0257
Epoch 3/20
37/37 [==============================] - 0s 667us/step - loss: 2.4088 - val_loss: 2.9809
Epoch 4/20
37/37 [==============================] - 0s 694us/step - loss: 2.3716 - val_loss: 2.9370
Epoch 5/20
37/37 [==============================] - 0s 722us/step - loss: 2.3349 - val_loss: 2.8932
Epoch 6/20
37/37 [==============================] - 0s 778us/step - loss: 2.2990 - val_loss: 2.8509
Epoch 7/20
37/37 [==============================] - 0s 667us/step - loss: 2.2639 - val_loss: 2.8086
Epoch 8/20
37/37 [==============================] - 0s 694us/step - loss: 2.2286 - val_loss: 2.7673
Epoch 9/20
37/37 [=

Epoch 18/20
37/37 [==============================] - 0s 722us/step - loss: 1.1901 - val_loss: 1.1909
Epoch 19/20
37/37 [==============================] - 0s 694us/step - loss: 1.1572 - val_loss: 1.1585
Epoch 20/20
10/10 [==============================] - 0s 444us/step - loss: 0.7686
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 0.8580 - val_loss: 1.0167
Epoch 2/20
37/37 [==============================] - 0s 694us/step - loss: 0.8361 - val_loss: 0.9932
Epoch 3/20
37/37 [==============================] - 0s 667us/step - loss: 0.8149 - val_loss: 0.9702
Epoch 4/20
37/37 [==============================] - 0s 694us/step - loss: 0.7943 - val_loss: 0.9476
Epoch 5/20
37/37 [==============================] - 0s 694us/step - loss: 0.7740 - val_loss: 0.9256
Epoch 6/20
37/37 [==============================] - 0s 694us/step - loss: 0.7543 - val_loss: 0.9045
Epoch 7/20
37/37 [==============================] - 0s 694us/step - loss: 0.7355 - val_loss: 0.8835
Epoch 8/20
37/37 [

...

Epoch 12/20
37/37 [==============================] - 0s 722us/step - loss: 1.2406 - val_loss: 1.3791
Epoch 13/20
37/37 [==============================] - 0s 722us/step - loss: 1.2321 - val_loss: 1.3705
Epoch 14/20
37/37 [==============================] - 0s 667us/step - loss: 1.2235 - val_loss: 1.3620
Epoch 15/20
37/37 [==============================] - 0s 667us/step - loss: 1.2151 - val_loss: 1.3535
Epoch 16/20
37/37 [==============================] - 0s 667us/step - loss: 1.2066 - val_loss: 1.3452
Epoch 17/20
37/37 [==============================] - 0s 694us/step - loss: 1.1983 - val_loss: 1.3369
Epoch 18/20
37/37 [==============================] - 0s 694us/step - loss: 1.1900 - val_loss: 1.3287
Epoch 19/20
37/37 [==============================] - 0s 667us/step - loss: 1.1818 - val_loss: 1.3205
Epoch 20/20
10/10 [==============================] - 0s 444us/step - loss: 1.1151
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 0.7164 - val_loss: 0.9847
Epoch 2/20
3

Epoch 11/20
37/37 [==============================] - 0s 667us/step - loss: 1.5441 - val_loss: 1.1314
Epoch 12/20
37/37 [==============================] - 0s 694us/step - loss: 1.5331 - val_loss: 1.1239
Epoch 13/20
37/37 [==============================] - 0s 694us/step - loss: 1.5222 - val_loss: 1.1164
Epoch 14/20
37/37 [==============================] - 0s 694us/step - loss: 1.5115 - val_loss: 1.1090
Epoch 15/20
37/37 [==============================] - 0s 667us/step - loss: 1.5006 - val_loss: 1.1017
Epoch 16/20
37/37 [==============================] - 0s 694us/step - loss: 1.4901 - val_loss: 1.0942
Epoch 17/20
37/37 [==============================] - 0s 694us/step - loss: 1.4794 - val_loss: 1.0869
Epoch 18/20
37/37 [==============================] - 0s 667us/step - loss: 1.4689 - val_loss: 1.0797
Epoch 19/20
37/37 [==============================] - 0s 694us/step - loss: 1.4585 - val_loss: 1.0725
Epoch 20/20
10/10 [==============================] - 0s 453us/step - loss: 0.7465
Epoch 1/2

C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
210 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
210 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "C:\Users\Benjamin\AppData\Lo

46/46 [==============================] - 0s 644us/step - loss: 0.0440 - val_loss: 0.0620
Epoch 3/20
46/46 [==============================] - 0s 644us/step - loss: 0.0375 - val_loss: 0.0636
Epoch 4/20
46/46 [==============================] - 0s 622us/step - loss: 0.0373 - val_loss: 0.0631
Epoch 5/20
46/46 [==============================] - 0s 622us/step - loss: 0.0371 - val_loss: 0.0626
Epoch 6/20
46/46 [==============================] - 0s 622us/step - loss: 0.0372 - val_loss: 0.0650
Epoch 7/20
46/46 [==============================] - 0s 644us/step - loss: 0.0372 - val_loss: 0.0633
Epoch 8/20
46/46 [==============================] - 0s 622us/step - loss: 0.0373 - val_loss: 0.0634
Epoch 9/20
46/46 [==============================] - 0s 622us/step - loss: 0.0373 - val_loss: 0.0628
Epoch 10/20
46/46 [==============================] - 0s 644us/step - loss: 0.0372 - val_loss: 0.0627
Epoch 11/20
46/46 [==============================] - 0s 622us/step - loss: 0.0373 - val_loss: 0.0646
Epoch 12/

['model.pkl']

In [40]:
print(rnd_mod.predict(X_val))
print(y_val)


6/6 [==============================] - 0s 400us/step
[3.22173188e+05 2.25001266e+05 8.86518062e+05 9.65657422e+04
 3.82751367e+04 4.33626406e+04 1.06862862e+06 7.38132375e+05
 2.05003812e+05 1.14154578e+05 1.71601062e+05 7.86469453e+04
 5.57873250e+05 2.49595016e+05 1.26708219e+05 8.65193516e+04
 8.86581484e+04 4.36993750e+04 2.48847312e+05 9.72053438e+04
 2.85688340e+04 3.87361312e+05 3.48661844e+05 1.57867234e+05
 4.53325430e+04 2.85012656e+04 2.83708625e+05 3.97406406e+05
 2.66449000e+05 1.72920922e+05 5.58282266e+04 7.41438750e+04
 1.12230738e+06 1.44179812e+05 5.80539961e+04 5.50393250e+05
 5.51621328e+04 1.48909262e+06 1.69694766e+04 4.32294648e+04
 2.27531328e+05 6.02465625e+04 6.02595742e+04 2.09464625e+05
 7.00938281e+04 6.91091125e+05 5.89702875e+05 6.84891094e+04
 3.37692281e+05 7.77701688e+05 2.93666219e+05 1.32645547e+05
 2.59367422e+04 1.23202094e+05 2.12884125e+06 4.15259031e+05
 1.91252344e+05 9.85553047e+04 3.22957438e+05 4.87821094e+04
 2.19667031e+05 4.92724727e+04 4

In [44]:
train_score = rnd_mod.score(X_train,y_train)
test_score = rnd_mod.score(X_test,y_test)
val_score = rnd_mod.score(X_val,y_val)


6/6 [==============================] - 0s 600us/step - loss: 7264400310272.0000


In [45]:
print("Train score: " + str(train_score ))
print("Test score: " + str(test_score))
print("Val score: " + str(val_score))

Train score: -0.036960944533348083
Test score: -0.06363268196582794
Val score: -7264400310272.0
